## スコアリングフェーズにおけるデータ処理（解決編）

前編にて、スコアリングデータのone-hotエンコーディング後に、以下の問題が生じる得ることを確認しました。<br>
- モデルデータにないカラムが生成される可能性
- モデルデータにあったカラムが消える可能性
- データ型の違いが理由で上記を生じさせてしまう可能性

そこでこの問題が生じないよう、コードを補強していくことにしましょう。前提として、私達はどのデータが連続変数で、どのデータがカテゴリ変数かは事前に把握しているとします。そして今回のカテゴリ変数は<b>Dependents, Gender, Married, Education, Self_Employed, Property_Areaの６変数</b>とします。最初の補強は、フラットファイルを読み込む段階で<b>カテゴリ変数をobject型として明記して読み込むこと</b>です。

In [ ]:
# import sample data: Loan screening data for classification 
import pandas as pd
df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv',
                 header=0,
                 dtype={'Dependents':object,
                        'Gender':object,
                        'Married':object,
                        'Education':object,
                        'Self_Employed':object,
                        'Property_Area':object})
X  = df.iloc[:,:-1]            # 最終列以前を特徴量
ID = X.iloc[:,[0]]             # 第0列はPK（Loan_ID）なのでIDとしてセット
X  = X.drop('Loan_ID', axis=1) # Loan_IDは特徴ベクトルから削除
y  = df.iloc[:,-1]             # 最終列を正解データ

# check the shape
print('---------------------------------')
print('Raw shape: (%i,%i)' %df.shape)
print('X shape: (%i,%i)' %X.shape)
print('---------------------------------')
print(X.dtypes)

# ローン審査でNOとなったサンプルを1（正例）として変換
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)
print('---------------------------------')
print(y.value_counts())

続けて、モデリング段階のone-hotエンコーディングを行います。ここに変更はありません。

In [ ]:
ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']

X_ohe = pd.get_dummies(X,
                       dummy_na=True,
                       columns=ohe_columns)

print('X_ohe shape:(%i,%i)' % X_ohe.shape)
X_ohe.head()

続けて、連続変数の欠損を平均値で置き換えます。ここも変更はありません。

In [ ]:
from sklearn.preprocessing import Imputer

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)

X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

X_ohe.head()

最後に、RFEによる特徴量選択を実施します。

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(random_state=1),
               n_features_to_select=10,
               step=.05)

selector.fit(X_ohe,y)

X_fin = X_ohe.loc[:, X_ohe_columns[selector.support_]]
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

以上でモデリング段階の処理が終わり、<br><b>ここからがテストデータを読み込みとなります。<br>ここでもカテゴリ変数の明示的な指定をしましょう。</b>

In [ ]:
import pandas as pd

# import sample data
# Loan screening data for classification 
df_s = pd.read_csv('./data/av_loan_test_Y3wMUE5_7gLdaTN.csv',
                   header=0,
                   dtype={'Dependents':object,
                          'Gender':object,
                          'Married':object,
                          'Education':object,
                          'Self_Employed':object,
                          'Property_Area':object})
ID_s = df_s.iloc[:,[0]]            # 第0列はPK（Loan_ID）なのでIDとしてセット
X_s  = df_s.drop('Loan_ID',axis=1) # Loan_IDは特徴ベクトルから削除

# check the shape
print('---------------------------------')
print('Raw shape: (%i,%i)' %df_s.shape)
print('X shape: (%i,%i)' %X_s.shape)
print('---------------------------------')
print(X_s.dtypes)
ID_s.join(X_s).head()

カテゴリ変数をきちんとデータ型でも揃えた状態で、one-hotエンコーディングを実施します。<br>Dependentsに見られた不整合が消えています。

In [ ]:
X_ohe_s = pd.get_dummies(X_s,
                         dummy_na=True,
                         columns=ohe_columns)
print('X_ohe_s shape:(%i,%i)' % X_ohe_s.shape)
X_ohe_s.head()

スコアリングのone-hotエンコーディングを終えたので、<br>この時点の特徴量リストをモデリング時とスコアリング時で再度比較してみます。

In [ ]:
cols_model = set(X_ohe.columns.values)
cols_score = set(X_ohe_s.columns.values)

# モデルにはあったがスコアにはないデータ項目
diff1 = cols_model - cols_score
print('モデルのみに存在する項目: %s' % diff1)

# スコアにはあるがモデルになかったデータ項目
diff2 = cols_score - cols_model
print('スコアのみに存在する項目: %s' % diff2)

データ型の違いによる不一致は消え、以下の違いだけが残りました。
- Dependents_3+はスコアリングデータには存在しない
- Gender_Unknownはスコアリングデータで新たに登場した

後者はモデリング時点にないカラムなので削除する他ありません。前者はモデル時点の再現のためデータ項目として再登場させなくてはいけません。それを実現する一つの方法がデータフレームを縦に結合するconcat処理です。一旦サンプルデータからはなれ、concatの基本動作を確認します。

In [ ]:
# カラム構成が同じデータフレームの結合
df1 = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
df2 = pd.DataFrame([[3,2,1]], columns=['c1','c2','c3'])
df_all = pd.concat([df1, df2])
df_all

In [ ]:
# カラム構成が異なるデータフレームの結合
df3 = pd.DataFrame([[0,1,2,3]],
                   columns=['c0','c1','c3','c4'])
df_all = pd.concat([df_all, df3])
df_all

上記のc0とc4は、元々あったデータフレーム（df_all）には存在しなかった項目という点で、スコアリングデータに初めて登場した項目と言えます。一方、c2はモデリングデータにはあったがスコアリングデータでは登場しなかったデータ項目となります。よって対応は、①c0とc4はドロップ、②c2はゼロ補完が妥当と結論されるものです。

それではサンプルデータに戻ります。<br>モデリング時点のone-hotエンコーディング処理後のカラム構成は、X_ohe_columnsでした。

In [ ]:
print(X_ohe_columns)

このカラム構成だけを持った（データ部分は持たない）データフレームを作ります。

In [ ]:
df_cols_m = pd.DataFrame(None,
                         columns=X_ohe_columns,
                         dtype=float)
df_cols_m

上記データフレームに対して、スコアリング時点のone-hotエンコーディング後のデータを縦に結合します。

In [ ]:
X_ohe_s2 = pd.concat([df_cols_m, X_ohe_s])
X_ohe_s2.head()
print(X_ohe_s2.shape)

次に、スコアリングデータのみに登場するデータ項目を削除しましょう。

In [ ]:
X_ohe_s2 = X_ohe_s2.drop(list(set(X_ohe_s.columns.values)-set(X_ohe.columns.values)),
              axis=1)
X_ohe_s2.head()
print(X_ohe_s2.shape)

次に、スコアリングでは登場しなかったデータ項目をゼロ埋めします。<br>Depend_3+がNaNから全てゼロとなりました。

In [ ]:
X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))] = \
  X_ohe_s2.loc[:,list(set(X_ohe.columns.values)-set(X_ohe_s.columns.values))].fillna(0, axis=1)
X_ohe_s2.head()

次に、モデリング時点のデータ項目の並び順を明示的に担保します。<br>以下の通り、reindex_axisを使うことで並び順を制御できます。

In [ ]:
test = pd.DataFrame([[1,2,3]], columns=['c1','c2','c3'])
display(test)
test = test.reindex(['c2','c3','c1'], axis=1)
display(test)

ここでは、モデリング時点のone-hotエンコーディング後の並び順に制御します。

In [ ]:
X_ohe_s2 = X_ohe_s2.reindex(X_ohe.columns.values, axis=1)
X_ohe_s2.head()

以上で、スコアリング時のone-hotエンコーディングをモデリング時の状況と合致させられました。<br><b>ここまで整合させたことでモデリング時に各変数の平均値を学習させたImputerが適用可能となります。</b><br>連続変数の欠損値も（学習時の）平均値で正しく置き換えられます。

In [ ]:
X_ohe_s3 = pd.DataFrame(imp.transform(X_ohe_s2),
                        columns=X_ohe_columns)
X_ohe_s3.head()

RFEによって選択された変数の位置はsupport_属性から取得できたので、<br>スコアリングデータの特徴量の最終形は以下のようになります。

In [ ]:
X_fin_s = X_ohe_s.loc[:,X_ohe_columns[selector.support_]]
print(X_fin_s.shape)
X_fin_s.head()

以上で、スコアリング段階におけるデータの前処理が終了です。<br>このデータを未知のXとして、学習済みモデルに入力すれば予測値を得ることができます。